In [2]:
'''Analisis of equivalency groups
on conflict basis'''
# take care about param types
import re
#import functions_is as fis

import numpy as np
import pandas as pd
import string # for int2base

from itertools import product
from itertools import combinations
import itertools
import timeit
import datetime
import copy
from collections import Counter

In [3]:
def listToString(list1):
    return (''.join(map(str, list1)))
def listToString_cmm(list1):
    return (','+','.join(map(str, list1))+',')

def list_getsome(vec, positions):
    list_n = []
    for y in positions:
        list_n.append(vec[y])
    #print("lnf", list_n)
    return(list_n)

def diversity_num(vec, dim):
    # return num of different digits without nans
    dvst = 0
    for i in range(0, dim):
        for j in range(0, len(vec)):
            if vec[j] == i:
                dvst += 1
                break
    return dvst
#len(set(tbl_merged[~np.isnan(tbl_merged[:,p]),p]))

def diversity_vec(vec, dim):
    dvst = []
    for i in range(0, dim):
        for j in range(0, len(vec)):
            if vec[j] == i:
                dvst.append(i)
                break
    return dvst

# diversity_vec_wnan # from collections import Counter
lst1 = Counter().keys() 
list()
    


[]

In [67]:
data = pd.read_csv('02_arctic_5x3_201223_data.csv', sep=',')
#print(data)
# data should contain only leaves and K columns, it is format
# test if there are duplicates 
#data.iloc[:,0:6].duplicated()

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
dtype: bool

In [4]:
# test if the set of examples can be realised 
# with incopmlete techniqe




#dim = 4
#load incomplete data
#data = pd.read_csv('feeley2012_cl_d4.csv', sep=',')
#data = pd.read_csv('210409_id_arctic_l5_datatable.csv', sep=',')
data = pd.read_csv('02_arctic_5x3_201223_data.csv', sep=';')
#print(data)
# data should contain only leaves and K columns, it is format
#data.duplicated()
lnum = data.shape[1]-1
print("lnum = ", lnum)
data = np.array(data.iloc[:,0:lnum+1])
#print("data: \n" ,data)
dim = 1+data.max()-data.min()
print("dim = ", (data.max()-data.min())+1)
#print("K: " ,data[:,lnum-1])
print("\n")

# all posible values of group numbers
lenumeration = [i for i in range(0,lnum)] 

len_combinations = 0
for i in range(2,lnum):
    lcombinations = list(combinations(range(0, lnum), i))
    len_combinations += len(lcombinations)

dfnumofisg = pd.DataFrame(
    {
        "name": np.array(len_combinations*2, dtype="str"),
        "val": np.empty(len_combinations*2, dtype="int32"),
    }
)

print("number of set | set on x leaves")
cnt = 0
for i in range(2,lnum):
    lcombinations = list(combinations(range(0, lnum), i))
    # posible combinatins of groups
    print(i-2, lcombinations)
    for i in range(len(lcombinations)):
        #print("lcombinations[i] ", lcombinations[i])
        #numofisg[cnt,0] = int(''.join(map(str, lcombinations[i])))
        dfnumofisg.iat[cnt,0] = ''.join(map(str, lcombinations[i]))
        dfnumofisg.iat[cnt,1] = 0
        #print("dfnumofisg[i] ", dfnumofisg.iloc[cnt,:])
        cnt += 1

# default values 
numofgroup = 2
numberofvalue = 1
# enter dialog
numofgroup = input('Enter number of set:') 
numofgroup = int(numofgroup)+2
numberofvalue = input('Enter number of value in set:') 
numberofvalue = int(numberofvalue)
print("\n")

lcombinations = list(combinations(range(0, lnum), numofgroup))
#print("lcombinations", lcombinations)

# form table K
print("group of leaves investigated: ", lcombinations[numberofvalue])
print("\n")
tbl_k = np.empty((dim**numofgroup,dim**(lnum-numofgroup)))
tbl_k.fill(np.nan)
colfilled = []
seen = set()
# fill table K from data string by sting (x)
for x in range(0,len(data)):
    #print(data[x,lcombinations[j]])
    # get coordinate in nondecimal representation of string
    data[x,lcombinations[numberofvalue]]
    # get coordinate in nondecimal representation of column
    reminder = list(set(lenumeration)-set(lcombinations[numberofvalue])) 
    data[x,reminder]
    # need to place F value in Ktable to selected coordinates
    c1 = "".join([str(xx) for xx in data[x,lcombinations[numberofvalue]]])
    c2 = "".join([str(xx) for xx in data[x,reminder]])
    c2_t = int(c2[::-1], base = dim)
    tbl_k[int(c1[::-1], base = dim), c2_t] = data[x,lnum]
    # попутно можно записать все столбцы в которых есть значения, соотв. другие м.б. удал.
    if c2_t not in seen:
        seen.add(c2_t)
        colfilled.append(c2_t) 
tbl_k_s = tbl_k[:,colfilled]
# del nan rows
rows_nan = np.all(np.isnan(tbl_k_s), axis = (1))
tbl_k_s = tbl_k_s[~rows_nan,:]
numbers = [int(i) for i in range(0,len(tbl_k_s))]
numbers_col = [int(i) for i in range(-1,tbl_k_s.shape[1])]
temp_k = np.hstack((np.array(numbers).reshape(len(tbl_k_s),1), tbl_k_s))
temp_k = np.vstack((np.array(numbers_col).reshape(1,temp_k.shape[1]), temp_k))
#print(tbl_k_s)
print(temp_k)

# for now it is necessary to analyze whether the given alphabet is enough
list_ind = [[i for i in range(tbl_k_s.shape[0])]]
inigroup = 1

for r in range(tbl_k_s.shape[1]):
    # select a column, from back because extend used further
    # checklist - digits in curr col
    #if len(set(tbl_k_s[:,r])) > 1:
    if diversity_num(tbl_k_s[:,r], dim) > 1:
        # if there is conflict of type 1 = more than 1 different digit in col
        #checklist = tbl_k_s[:,r]
        checklist = copy.deepcopy(tbl_k_s[:,r])
        checklist_pos = 0
        for z in range(len(list_ind)-1,-1,-1): 
            # select a range in list_ind
            # rev list - это связано с тем что del удаляет с конца
            # в этой версии это не нужно
            ind_t = []
            flag_del_inigroup = 0
            nan_counted = 0
            for w in range(dim):
                # search for every simbol in alphabet in column                      
                range_in_range = np.where(tbl_k_s[list_ind[z],r] == w)[0].tolist()
                #if r > 0 and len(range_in_range) and not nan_counted:
                if not inigroup and len(range_in_range) and not nan_counted:
                    # inigroup for don't count nans when inigroup activated
                    plus_na = np.argwhere(np.isnan(tbl_k_s[list_ind[z],r]))
                    plus_na = list(itertools.chain(*plus_na))
                    range_in_range = list(itertools.chain(*[range_in_range, plus_na]))
                    nan_counted = 1
                if len(range_in_range):
                    range_in_range = list_getsome(list_ind[z], range_in_range)
                    checklist[range_in_range] = np.nan
                    ind_t.append(range_in_range)        
                    flag_del_inigroup = 1

            if flag_del_inigroup:
                del list_ind[z]
                flag_del_inigroup = 0
                inigroup = 0
            # sort list for further includes and dubles checking
            for c in range(len(ind_t)):
                ind_t[c].sort()
            list_ind.extend(ind_t)
        # seek for opportunity to expand eqg by new digits
        # checklist_pos - digits in checklist
        if ~np.isnan(checklist).all():
            # test if there are not nans
            # каждую позицию из разнообразия нужно приклеить без противоречия
            # берем доступные числа из групп столбца и расширяем группы по ним
            checklist_pos = np.argwhere(~np.isnan(checklist))#[0].tolist()
            checklist_pos = list(itertools.chain(*checklist_pos))                
            # need to sort list_ind for groups with digits go first
            zzt = 0
            cnt_zzt = 0
            while cnt_zzt < len(list_ind)-1:
                if not len(set(tbl_k_s[list_ind[zzt],r])):
                    print(tbl_k_s[list_ind[zzt],r])
                    print(len(set(tbl_k_s[list_ind[zzt],r])))
                    list_ind.append(list_ind[zzt])
                    del list_ind[zzt]
                else: zzt += 1
                cnt_zzt += 1
            for zz in range(len(list_ind)):
                ind_tt = list_ind[zz]
                for j in checklist[checklist_pos]:
                    j = int(j)
                    ldiv = diversity_vec(tbl_k_s[ind_tt,r], dim)
                    cnt_j = 0 
                    # проверка ГЭ на наличие символа отличного от того для которого мы ищем группу                           
                    # сформировать список без j и искать вхождение из ldiv в него
                    list_without_j = [i for i in range(dim)]
                    del list_without_j[np.nonzero(np.array(list_without_j) == j)[0][0]]
                    if any(item in ldiv for item in list_without_j):
                        continue
                    else:
                        list_ind[zz].append(checklist_pos[cnt_j])
                        checklist[checklist_pos[cnt_j]] = np.nan
                        checklist_pos = np.argwhere(~np.isnan(checklist))#[0].tolist()
                        checklist_pos = list(itertools.chain(*checklist_pos))
                    cnt_j += 1
        #else # если в ГЭ нет номера из checklist, то добваить группу
        # проверить остались ли свободные числа in checklist
        if ~np.isnan(checklist).all():
            # для каждого отдельного номинала своя группа
            ldiv2 = diversity_vec(checklist, dim)
            for i in ldiv2:
                range_in_range2 = np.where(checklist == i)[0].tolist()
                list_ind.append(range_in_range2)

print("\n")
print("groups found = ", len(list_ind))
print("final partition: ", list_ind)      

   

lnum =  5
dim =  3


number of set | set on x leaves
0 [(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]
1 [(0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 2, 3), (0, 2, 4), (0, 3, 4), (1, 2, 3), (1, 2, 4), (1, 3, 4), (2, 3, 4)]
2 [(0, 1, 2, 3), (0, 1, 2, 4), (0, 1, 3, 4), (0, 2, 3, 4), (1, 2, 3, 4)]
Enter number of set:0
Enter number of value in set:0


group of leaves investigated:  (0, 1)


[[-1.  0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.]
 [ 0.  1. nan nan nan nan nan nan nan nan  0. nan nan nan nan nan nan]
 [ 1.  1. nan nan  1. nan nan nan nan  1. nan nan nan  1. nan  1. nan]
 [ 2. nan nan nan nan  2. nan nan nan nan nan  2. nan  1. nan nan nan]
 [ 3.  2. nan nan nan  2. nan nan  1. nan nan nan nan nan nan nan nan]
 [ 4. nan  2. nan  2.  2.  2. nan  1. nan nan nan  1. nan  2. nan  1.]
 [ 5. nan nan  2. nan  2. nan nan nan nan  1. nan nan nan nan nan nan]
 [ 6. nan nan nan nan nan nan nan  1. nan nan nan nan nan nan nan nan]
 [ 7. nan nan  